###Instalação do Postgres

In [ ]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS enterprise;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE enterprise;'

In [ ]:
%env TFIO_DEMO_DATABASE_NAME=enterprise
%env TFIO_DEMO_DATABASE_HOST=localhost
%env TFIO_DEMO_DATABASE_PORT=5432
%env TFIO_DEMO_DATABASE_USER=postgres
%env TFIO_DEMO_DATABASE_PASS=postgres

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!PGPASSWORD=$TFIO_DEMO_DATABASE_PASS psql -q -h $TFIO_DEMO_DATABASE_HOST -p $TFIO_DEMO_DATABASE_PORT -U $TFIO_DEMO_DATABASE_USER -d $TFIO_DEMO_DATABASE_NAME -f '/content/northwind.sql'
!psql postgres < '/content/drive/MyDrive/modulo4/northwind.sql'

In [ ]:
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


'Connected: @postgres'

In [ ]:
#Exemplo de consulta sql
df = %sql SELECT * FROM public.orders
df.head()

###Imports

In [ ]:
#Já estão instalados:
#!pip install psycopg2
#!pip install sqlalchemy

In [ ]:
import pandas as pd
import numpy as np

#import sqlalchemy as db
from sqlalchemy import create_engine

#Já estão instalados:
#!pip install psycopg2
#import psycopg2


###Consultas de aula prática 1

####Consulta 1

Testar as seguintes consultas:
1. Os 10 produtos mais caros

select distinct products.productname as tenmostexpensiveproducts, products.unitprice from products order by products.unitprice desc;

In [ ]:
#Questão 1
df_q1 = %sql SELECT DISTINCT public.products.product_name AS tenmostexpensiveproducts, public.products.unit_price FROM public.products ORDER BY public.products.unit_price DESC
df_q1

 * postgresql+psycopg2://@/postgres


,tenmostexpensiveproducts,unit_price
0,Côte de Blaye,263.50
1,Thüringer Rostbratwurst,123.79
2,Mishi Kobe Niku,97.00
3,Sir Rodney's Marmalade,81.00
4,Carnarvon Tigers,62.50
...,...,...
72,Tourtière,7.45
73,Filo Mix,7.00
74,Konbu,6.00
75,Guaraná Fantástica,4.50


###Consulta 2

select customers.customerid, customers.companyname, customers.city, customers.country
from customers right join orders on customers.customerid = orders.customerid
where (((orders.orderdate) between #1/1/1995# and #12/31/1995#));


In [ ]:
#Pedido Trimestral
df_q2 = %sql SELECT public.customers.customer_id, public.customers.company_name, public.customers.city, public.customers.country FROM public.customers RIGHT JOIN public.orders ON public.customers.customer_id = public.orders.customer_id WHERE (((public.orders.order_date) BETWEEN '1995-01-01' AND '1995-12-31'))

if (df_q2.empty == True):
  print("Consulta sem resposta")
else:
  df_q2

 * postgresql+psycopg2://@/postgres
Consulta sem resposta


####Consulta 3

3. Detalhes do pedido estendido

select distinctrow [order details].orderid, [order details].productid, products.productname, [order details].unitprice, [order details].quantity, [order details].discount, ccur([order details].[unitprice]*[quantity]*(1-[discount])/100)*100 as extendedprice
from products inner join [order details] on products.productid = [order details].productid
order by [order details].orderid;

In [ ]:
#(psycopg2.ProgrammingError) function ccur(double precision) does not exist
#ccur() foi substituído por numpy.float128()

df_q3 = %sql SELECT DISTINCT public.order_details.order_id, public.order_details.product_id, public.products.product_name, public.order_details.unit_price, public.order_details.quantity, public.order_details.discount, (ROUND(order_details.unit_price*order_details.quantity*(1-order_details.discount)/100)*100) AS extendedprice FROM public.products INNER JOIN public.order_details ON public.products.product_id = public.order_details.product_id ORDER BY public.order_details.order_id

df_q3

 * postgresql+psycopg2://@/postgres


,order_id,product_id,product_name,unit_price,quantity,discount,extendedprice
0,10248,11,Queso Cabrales,14.00,12,0.00,200.0
1,10248,42,Singaporean Hokkien Fried Mee,9.80,10,0.00,100.0
2,10248,72,Mozzarella di Giovanni,34.80,5,0.00,200.0
3,10249,14,Tofu,18.60,9,0.00,200.0
4,10249,51,Manjimup Dried Apples,42.40,40,0.00,1700.0
...,...,...,...,...,...,...,...
2150,11077,64,Wimmers gute Semmelknödel,33.25,2,0.03,100.0
2151,11077,66,Louisiana Hot Spiced Okra,17.00,1,0.00,0.0
2152,11077,73,Röd Kaviar,15.00,2,0.01,0.0
2153,11077,75,Rhönbräu Klosterbier,7.75,4,0.00,0.0


In [ ]:

resultado = %sql SELECT ROUND(248.6759, 0)
resultado

 * postgresql+psycopg2://@/postgres


,round
0,249


####Consulta 4

4. Produtos por baixo do valor médio

select distinctrow products.productname, products.unitprice
from products
where (((products.unitprice)>(select avg([unitprice]) from products)))
order by products.unitprice desc;

In [ ]:
df_q4 = %sql SELECT DISTINCT public.products.product_name, public.products.unit_price FROM public.products WHERE (((public.products.unit_price)>(SELECT AVG(public.products.unit_price) FROM public.products))) ORDER BY public.products.unit_price DESC
df_q4

 * postgresql+psycopg2://@/postgres


,product_name,unit_price
0,Côte de Blaye,263.50
1,Thüringer Rostbratwurst,123.79
2,Mishi Kobe Niku,97.00
3,Sir Rodney's Marmalade,81.00
4,Carnarvon Tigers,62.50
5,Raclette Courdavault,55.00
6,Manjimup Dried Apples,53.00
7,Tarte au sucre,49.30
8,Ipoh Coffee,46.00
9,Rössle Sauerkraut,45.60


####Consulta 5

5. Subtotal do pedido
select distinctrow [order details].orderid, sum(ccur([unitprice]*[quantity]*(1-[discount])/100)*100) as subtotal
from [order details]
group by [order details].orderid;

In [ ]:
df_q5 = %sql SELECT DISTINCT public.order_details.order_id, SUM(ROUND(public.order_details.unit_price*public.order_details.quantity*(1-public.order_details.discount)/100)*100) AS subtotal FROM public.order_details GROUP BY PUBLIC.order_details.order_id
df_q5

 * postgresql+psycopg2://@/postgres


,order_id,subtotal
0,10256,500.0
1,10294,2000.0
2,10344,2300.0
3,11075,500.0
4,11054,300.0
...,...,...
825,10297,1500.0
826,10541,1900.0
827,10735,500.0
828,10989,1300.0


####Consulta 6

Vendas por ano 

####Consulta 7

 Produtos vendidos em 1995

####Consulta 8

Vendas por Categoria